# Can Neural Networks detect Dynamical Phase Transitions ?

Can the recurrent neural network protocol developed by Evert et. al \cite{Evert2018PRB} detect dynamical phase transitions ? There are several questions one can probe here:
\begin{enumerate}
\item Does the RNN correctly predict the phase transition? Quenches across the critical point have non-analyticities.
\item What do the Neurons learn? Can they predict the time at which the non-analyticities appear ?
\end{enumerate}

# Code to simulate NN Transverse Field Ising model
We have the following Hamiltonian
\begin{align}
H&=-J\sum_{i} \sigma_{i}^{x}\sigma_{i+1}^{x}-\Gamma\sum_{i}\sigma_{i}^{z}
\end{align}

Now we recall the main results for diagonalizing the TFIM. With the Jordan-Wigner Transform, we get
\begin{align}
H_{\eta}=J\sum_{i=1}^{L-1}\left(a_{\eta,i}-a_{\eta,i}^{\dagger}\right)\left(a_{\eta,i+1}+a_{\eta,i+1}^{\dagger}\right)+B\sum_{i=1}^{L}\left(2a_{\eta,i}^{\dagger}a_{\eta,i}-1\right)+\eta\left(a_{\eta,L}-a_{\eta,L}^{\dagger}\right)\left(a_{\eta,1}+a_{\eta,1}^{\dagger}\right)
\end{align}
where, $\eta=\pm 1$ denotes the even and odd parity sector. We will focus largely on $\eta=1$.

Now, we diagonalize the above Hamiltonian to obtain the dispersion,
\begin{align}
H_{\eta}&=\frac{\Gamma}{2}+\sum_{q_{\eta}>0}\omega_{q_{\eta}}\left(\gamma_{\eta,q_{\eta}}^{\dagger}\gamma_{\eta,q_{\eta}}+\gamma_{\eta,-q_{\eta}}^{\dagger}\gamma_{\eta,-q_{\eta}}-1\right)\\
\gamma_{\eta,q_{\eta}}&=u_{q_{\eta}}b_{\eta,q{}_{\eta}}+iv_{q_{\eta}}b_{\eta,-q{}_{\eta}}^{\dagger}\\
    \omega_{q_{\eta}}&=2\sqrt{\Gamma^{2}-\Gamma J\cos q_{\eta}+J^{2}} \\
|\phi_{\rm GS}\rangle&=\frac{1}{\left|v_{q_{\eta}}\right|}\prod_{q_{\eta}>0}\gamma_{q_{\eta}}\gamma_{-q_{\eta}}|0\rangle\\
 u_{q_{\eta}}=\cos\theta_{q_{\eta}},\ \  v_{q_{\eta}}=\sin\theta_{q_{\eta}}& \ \ \ \tan2\theta_{q_{\eta}}=\frac{J\sin q_{\eta}}{\Gamma-J\cos q_{\eta}}
\end{align}
with, $q_{\eta}=\left(2m+\frac{1}{2}\left(\eta-1\right)\right)\frac{\pi}{L}$.   with $\eta=1$

 \underline{Quench Dynamics}

Now, to discuss quench dynamics from $\Gamma_0 \rightarrow \Gamma$. Let the angle change from $\theta_{q0}\rightarrow \theta_q$. Defining $\alpha_q=\theta_q-\theta_{q0}$, we have the initial state as the ground state.
\begin{align}
|\psi_{\rm in}\rangle=\prod_{q>0}(\cos\alpha_q+i\sin\alpha_q \gamma^\dagger_q\gamma^\dagger_{-q})|\phi_{\rm GS}\rangle
\end{align}


The Loschmidt Echo is given as,
\begin{align}
\Lambda(t)=\left|\left(\langle \psi_{\rm in} |\exp (-iHt)|\psi_{\rm in}\rangle\right)\right|^2 = \prod_{q>0} [1-\sin^2(2\alpha_q)\sin^2(\omega_qt)]
\end{align}

In [2]:
import numpy as np
from scipy import linalg as LA
import scipy.sparse.linalg as spla
from scipy import optimize as OPT
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm 
from latex_envs.latex_envs import figcaption
from scipy.special import binom as bm
from sympy.physics.quantum.cg import CG
import numpy.matlib
#import chardet
import os
import h5py
#for Latex font on figures
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Roman']})
rc('text', usetex=True)

/home/paraj/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
####################Function Definitions#############
def Loschmidt_Echo_TFIM()

In [19]:
######################Parameter definitions###################
#Global definitions
L=100 #System size
η=1 #parity sector

#Initial state
J0=1
Γ=0
#quench Hamiltonian
J=1 
Γ=2
T=100.

#define arrays
qarr=(2*np.arange(0,L/2)+0.5*(η-1))*np.pi/L # array with only positive q values
tarr=np.linspace(0,T,100)
Λ=np.zeros(np.size(tarr))
ωqarr= 2*np.sqrt(Γ**2+J**2-Γ*J*(np.cos(qarr)))

In [14]:
for t in tarr:
    for q in qarr:
        Λ[]

In [16]:
1+np.cos(qarr)

array([0.00000000e+00, 1.97327157e-03, 7.88529869e-03, 1.77127493e-02,
       3.14168389e-02, 4.89434837e-02, 7.02235141e-02, 9.51729475e-02,
       1.23693320e-01, 1.55672074e-01, 1.90983006e-01, 2.29486757e-01,
       2.71031373e-01, 3.15452894e-01, 3.62576010e-01, 4.12214748e-01,
       4.64173205e-01, 5.18246326e-01, 5.74220708e-01, 6.31875447e-01,
       6.90983006e-01, 7.51310113e-01, 8.12618685e-01, 8.74666766e-01,
       9.37209480e-01, 1.00000000e+00, 1.06279052e+00, 1.12533323e+00,
       1.18738131e+00, 1.24868989e+00, 1.30901699e+00, 1.36812455e+00,
       1.42577929e+00, 1.48175367e+00, 1.53582679e+00, 1.58778525e+00,
       1.63742399e+00, 1.68454711e+00, 1.72896863e+00, 1.77051324e+00,
       1.80901699e+00, 1.84432793e+00, 1.87630668e+00, 1.90482705e+00,
       1.92977649e+00, 1.95105652e+00, 1.96858316e+00, 1.98228725e+00,
       1.99211470e+00, 1.99802673e+00, 2.00000000e+00, 1.99802673e+00,
       1.99211470e+00, 1.98228725e+00, 1.96858316e+00, 1.95105652e+00,
      

# References

[<a id="cit-Evert2018PRB" href="#call-Evert2018PRB">1</a>] van Nieuwenburg Evert, Bairey Eyal and Refael Gil, ``_Learning phase transitions from dynamics_'', Phys. Rev. B, vol. 98, number , pp. 060301, Aug 2018.  [online](https://link.aps.org/doi/10.1103/PhysRevB.98.060301)

